ETL (Extracción, Transformación y Carga) es un proceso en el que se extraen datos de una fuente, se transforman y se cargan en otra fuente. En este caso, se extraen datos de un archivo json, se transforman y se cargan en un archivo csv.

In [112]:
# Cargamos las librerias necesarias para el desarrollo del ETL.
import pandas as pd
import numpy as np
import gzip
import ast
from dateutil import parser
from textblob import TextBlob

Creamos una función que se encargue de leer el archivo json y devolver un dataframe de pandas.

In [113]:
# Funcion para convertir un archivo .gz a un dataframe.
def gzip_to_df(file_path):                                          
    with gzip.open(file_path, 'rt', encoding='utf-8-sig') as file:  # Abrimos el archivo .gz
        return [ast.literal_eval(line) for line in file]            # Retornamos el archivo .gz como un dataframe.

Iniciamos con el procesamientos de los datos de user_reviews.json. Para ello, hacemos uso de la función creada anteriormente y la guardamos en una variable llamada df_reviews.

In [114]:
# Cargamos los datos de los reviews y los convertimos en un dataframe.
data = gzip_to_df('./DATA/DATA ORIGINAL/user_Reviews.json.gz')     # Cargamos los datos de los reviews.
df_reviews = pd.DataFrame(data)                                    # Convertimos los datos de los reviews en un dataframe.
df_reviews.head()                                                  # Mostramos los primeros 5 registros del dataframe.

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [115]:
df_reviews.info() # Vemos la informacion del dataframe.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


Vemos que la columna 'reviews' contiene un diccionario con los datos de cada review. Para poder acceder a los datos de cada review, creamos una función que se encargue de extraer los datos de cada review y devolver un dataframe de pandas.

In [116]:
# Desanidamos los datos de la columna reviews.
def extraer_values(df, columna):
    # Lista para almacenar los datos desanidados
    extracted_rows = []

    for index, row in df.iterrows(): # Itera sobre cada fila del DataFrame
        user_id = row["user_id"] 
        columnai = row[columna]
        
        if isinstance(columnai, list):
            for item in columnai:
                extracted_row = {"user_id": user_id}
                extracted_row.update(item)  # Agrega todas las claves y valores de 'item' al diccionario
                extracted_rows.append(extracted_row)

    # Crea un DataFrame a partir de la lista de datos desanidados
    df_new = pd.DataFrame(extracted_rows)
    return df_new

# Nombre de la columna a desanidar
columna = "reviews"

# Llama a la función con el DataFrame df_reviews
df_reviews_desanidado = extraer_values(df_reviews, columna)

df_reviews_desanidado.head() # Mostramos los primeros 5 registros del dataframe.

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


La columna 'posted' contiene la fecha en la que se publicó la review. Todas las fechas son diferentes, entonces creamos una función que se encargue de convertir la fecha a un formato más legible.

In [117]:
# Cambiamos el formato de fecha de la columna posted.
date_string = df_reviews_desanidado['posted'] # Obtiene la columna "posted"

def transform_date(date_string): # Función para transformar la fecha
    try:
        if isinstance(date_string, str):
            # Elimina "Posted" del principio y analiza la fecha
            date_string = date_string.replace('Posted', '').strip()
            parsed_date = parser.parse(date_string, fuzzy=True)
            return parsed_date
    except ValueError:
        pass  # Ignora fechas incorrectas
    return None  # Devuelve None para fechas inválidas o nulas

# Aplica la función de transformación a la columna "posted"
df_reviews_desanidado['posted'] = df_reviews_desanidado['posted'].apply(transform_date)

# Elimina las filas con valores nulos en la columna 'posted'
df_reviews_final = df_reviews_desanidado.dropna(subset=['posted'])

# Muestra el DataFrame resultante
df_reviews_final.head() 

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [118]:
# Cuenta la cantidad de valores nulos en la columna 'posted'
cantidad_valores_nulos = df_reviews_final['posted'].isnull().sum()
print("Cantidad de valores nulos en 'posted':", cantidad_valores_nulos) 

Cantidad de valores nulos en 'posted': 0


In [119]:
# Utiliza el método merge para combinar los DataFrames en base a la columna 'user_id'
df_reviews_final = df_reviews.merge(df_reviews_final, on='user_id', how='inner')
df_reviews_final.head()

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...",,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...",,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [120]:
# Vemos la informacion del dataframe final.
df_reviews_final.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61053 entries, 0 to 61052
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      61053 non-null  object        
 1   user_url     61053 non-null  object        
 2   reviews      61053 non-null  object        
 3   funny        61053 non-null  object        
 4   posted       61053 non-null  datetime64[ns]
 5   last_edited  61053 non-null  object        
 6   item_id      61053 non-null  object        
 7   helpful      61053 non-null  object        
 8   recommend    61053 non-null  bool          
 9   review       61053 non-null  object        
dtypes: bool(1), datetime64[ns](1), object(8)
memory usage: 4.3+ MB


ANALISIS DE SENTIMIENTOS: Nos piden realizar un análisis de sentimientos de las reviews. Para ello, hacemos uso de la librería TextBlob. Creamos una función que se encargue de analizar el sentimiento de cada review y devolver un dataframe de pandas.

In [121]:
# Creamos una funcion para el analisis de sentimientos.
def analisis_sentimientos(review):
    if isinstance(review, list) and len(review) > 0:    # Si es una lista y tiene mas de 0 elementos.
        text = review[0].get('review', '')              # Obtenemos el texto del review.
        sentimiento = TextBlob(text).sentiment.polarity # Obtenemos el sentimiento del texto.
        
        if sentimiento < -0.2:
            return 0 # Negativo 
        elif sentimiento >= -0.2 and sentimiento <= 0.2:
            return 1 # Neutral
        else:
            return 2 # Positivo
    else:
        return 1 # Neutral para valores nulos

In [122]:
df_reviews_final['sentiment_analysis'] = df_reviews_final['reviews'].apply(analisis_sentimientos) # Aplicamos la funcion al dataframe.
df_reviews_final.head() # Mostramos los primeros 5 registros del dataframe.

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,1
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.,1
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,1
3,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...",,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,1
4,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...",,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,1


In [123]:
# Eliminamos las columnas que no necesitamos
columnas_a_eliminar = ['reviews', 'funny', 'last_edited', 'helpful', 'review'] # Columnas a eliminar
df_reviews_final.drop(columnas_a_eliminar, axis=1, inplace=True)               # Eliminamos las columnas
df_reviews_final.head()

,user_id,user_url,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,True,1
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-07-15,22200,True,1
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-04-21,43110,True,1
3,js41637,http://steamcommunity.com/id/js41637,2014-06-24,251610,True,1
4,js41637,http://steamcommunity.com/id/js41637,2013-09-08,227300,True,1


In [124]:
df_reviews_final.info() # Vemos la informacion del dataframe final.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61053 entries, 0 to 61052
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_id             61053 non-null  object        
 1   user_url            61053 non-null  object        
 2   posted              61053 non-null  datetime64[ns]
 3   item_id             61053 non-null  object        
 4   recommend           61053 non-null  bool          
 5   sentiment_analysis  61053 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int64(1), object(3)
memory usage: 2.4+ MB


In [125]:
# Guardamos el dataframe final en un archivo csv.
df_reviews_final.to_csv('./DATA/DATA PROCESADA/df_reviews_final.csv', index=False) 

Iniciamos con el procesamiento de los datos de user_items.json. Para ello, hacemos uso de la función creada anteriormente y la guardamos en una variable llamada df_items.

In [126]:
# Cargamos los datos de los items y los convertimos en un dataframe, usando la funcion creada anteriormente.
data = gzip_to_df('./DATA/DATA ORIGINAL/users_items.json.gz')  # Cargamos los datos de los items.
df_items = pd.DataFrame(data)                                  # Convertimos los datos de los items en un dataframe.
df_items.head()                                                # Mostramos los primeros 5 registros del dataframe.

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Vemos que la columna 'items' contiene un diccionario con los datos de cada item. Para poder acceder a los datos de cada item, creamos una función que se encargue de extraer los datos de cada item y devolver un dataframe de pandas.

In [127]:
# Desanidamos los datos de la columna items.
def extraer_values(df, columna):
    # Lista para almacenar los datos desanidados
    extracted_rows = []

    for index, row in df.iterrows():
        user_id = row["user_id"]
        columnai = row[columna]
        
        if isinstance(columnai, list):
            for item in columnai:
                extracted_row = {"user_id": user_id}
                extracted_row.update(item)  # Agrega todas las claves y valores de 'item' al diccionario
                extracted_rows.append(extracted_row)

    # Crea un DataFrame a partir de la lista de datos desanidados
    df_new = pd.DataFrame(extracted_rows)
    return df_new

# Nombre de la columna a desanidar
columna = "items"

# Llama a la función con el DataFrame df_reviews
df_items_desanidado = extraer_values(df_items, columna)

df_items_desanidado.head() # Mostramos los primeros 5 registros del dataframe.

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0


In [128]:
# Utiliza el método merge para combinar los DataFrames en base a la columna 'user_id'
df_items_final = df_items.merge(df_items_desanidado, on='user_id', how='inner') # Combinamos los dataframes.
df_items_final.head() # Mostramos los primeros 5 registros del dataframe.

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",30,Day of Defeat,7,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",40,Deathmatch Classic,0,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",50,Half-Life: Opposing Force,0,0


In [129]:
df_items_final.drop(columns=['items', 'playtime_2weeks'], inplace=True) # Eliminamos las columnas que no necesitamos

In [130]:
df_items_final = df_items_final.dropna() # Eliminamos las filas con valores nulos
df_items_final = df_items_final.dropna(how='all')

In [131]:
df_items_final.info() # Vemos la informacion del dataframe final.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5272423 entries, 0 to 5272422
Data columns (total 7 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   steam_id          object
 3   user_url          object
 4   item_id           object
 5   item_name         object
 6   playtime_forever  int64 
dtypes: int64(2), object(5)
memory usage: 281.6+ MB


In [132]:
# Eliminamos filas sin informacion.
df_items_final = df_items_final.drop(df_items_final.index[799000:5272423]) 

In [133]:
df_items_final.info() # Vemos la informacion del dataframe final.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799000 entries, 0 to 798999
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           799000 non-null  object
 1   items_count       799000 non-null  int64 
 2   steam_id          799000 non-null  object
 3   user_url          799000 non-null  object
 4   item_id           799000 non-null  object
 5   item_name         799000 non-null  object
 6   playtime_forever  799000 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 42.7+ MB


In [134]:
df_items_final.to_csv('./DATA/DATA PROCESADA/df_items_final.csv', index=False) # Guardamos el dataframe final en un archivo csv.

Iniciamos el procesamiento de los datos de user_games.json. Para ello, hacemos uso de la función creada anteriormente y la guardamos en una variable llamada df_games.

In [135]:
# Cargamos los datos de los juegos y los convertimos en un dataframe.
with gzip.open('./DATA/DATA ORIGINAL/steam_games.json.gz', 'rt', encoding='utf-8-sig') as file: # Abrimos el archivo .gz
    data = file.readlines() # Leemos el archivo .gz

data = map(lambda x: x.rstrip(), data) # Eliminamos los espacios en blanco
data_json_str = "[" + ','.join(data) + "]" # Convertimos los datos en un string json
df_games = pd.read_json(data_json_str) # Convertimos los datos en un dataframe
df_games.head() # Mostramos los primeros 5 registros del dataframe.

C:\Users\jhgal\AppData\Local\Temp\ipykernel_24408\3290872402.py:7: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_games = pd.read_json(data_json_str) # Convertimos los datos en un dataframe


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [136]:
df_games.info() # Vemos la informacion del dataframe.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [137]:
df_games['release_date'] = df_games['release_date'].astype(str) # Convertimos la columna 'release_date' a string

In [138]:
# Cambiamos el formato de fecha de la columna posted.
date_string = df_games['release_date']

def transform_date(date_string): # Función para transformar la fecha
    try:
        if isinstance(date_string, str):
            # Elimina "Posted" del principio y analiza la fecha
            date_string = date_string.replace('release_date', '').strip()
            parsed_date = parser.parse(date_string, fuzzy=True)
            return parsed_date
    except ValueError:
        pass  # Ignora fechas incorrectas
    return None  # Devuelve None para fechas inválidas o nulas

# Aplica la función de transformación a la columna "posted"
df_games['release_date'] = df_games['release_date'].apply(transform_date)

# Elimina las filas con valores nulos en la columna 'posted'
df_games_final = df_games.dropna(subset=['release_date'])

# Muestra el DataFrame resultante
df_games_final.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0.0,772540.0,Trickjump Games Ltd


In [139]:
# Elimino los valores nulos.
df_games_final.dropna(how='all', inplace=True) 
df_games_final.head()

C:\Users\jhgal\AppData\Local\Temp\ipykernel_24408\3076319695.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_final.dropna(how='all', inplace=True)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0.0,772540.0,Trickjump Games Ltd


In [140]:
# Verificamos la cantidad de valores nulos en el dataframe.
# Verificar la cantidad total de filas en el DataFrame
total_filas = df_games_final.shape[0] 

# Verificar la cantidad de filas no nulas en el DataFrame
filas_no_nulas = df_games_final.dropna().shape[0]

# Calcular la cantidad de filas vacías o sin información
filas_sin_informacion = total_filas - filas_no_nulas

print(f"Total de filas: {total_filas}")
print(f"Filas sin información: {filas_sin_informacion}")

Total de filas: 29964
Filas sin información: 7436


In [141]:
df_games_final['id'] = df_games_final['id'].fillna(0).astype(int) # Convertimos la columna 'id' a int

C:\Users\jhgal\AppData\Local\Temp\ipykernel_24408\3001984210.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_final['id'] = df_games_final['id'].fillna(0).astype(int) # Convertimos la columna 'id' a int


In [142]:
df_games_final.info() # Vemos la informacion del dataframe final.

<class 'pandas.core.frame.DataFrame'>
Index: 29964 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   publisher     23966 non-null  object        
 1   genres        28730 non-null  object        
 2   app_name      29963 non-null  object        
 3   title         29963 non-null  object        
 4   url           29964 non-null  object        
 5   release_date  29964 non-null  datetime64[ns]
 6   tags          29803 non-null  object        
 7   reviews_url   29963 non-null  object        
 8   specs         29295 non-null  object        
 9   price         28818 non-null  object        
 10  early_access  29964 non-null  float64       
 11  id            29964 non-null  int32         
 12  developer     28714 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(10)
memory usage: 3.1+ MB


In [143]:
df_games_final['early_access'] = df_games_final['early_access'].astype(int) # Convertimos la columna 'early_access' a int

C:\Users\jhgal\AppData\Local\Temp\ipykernel_24408\2332968529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_final['early_access'] = df_games_final['early_access'].astype(int) # Convertimos la columna 'early_access' a int


In [144]:
df_games_final.dropna(inplace=True) # Eliminamos las filas con valores nulos

C:\Users\jhgal\AppData\Local\Temp\ipykernel_24408\3464952241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_final.dropna(inplace=True) # Eliminamos las filas con valores nulos


In [145]:
# Verificamos la cantidad de valores nulos en el dataframe.
# Verificar la cantidad total de filas en el DataFrame
total_filas = df_games_final.shape[0]

# Verificar la cantidad de filas no nulas en el DataFrame
filas_no_nulas = df_games_final.dropna().shape[0]

# Calcular la cantidad de filas vacías o sin información
filas_sin_informacion = total_filas - filas_no_nulas

print(f"Total de filas: {total_filas}")
print(f"Filas sin información: {filas_sin_informacion}")

Total de filas: 22528
Filas sin información: 0


In [146]:
df_games_final.head() # Mostramos los primeros 5 registros del dataframe.

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0,767400,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0,772540,Trickjump Games Ltd


In [147]:
df_games_final.info() # Vemos la informacion del dataframe final.

<class 'pandas.core.frame.DataFrame'>
Index: 22528 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   publisher     22528 non-null  object        
 1   genres        22528 non-null  object        
 2   app_name      22528 non-null  object        
 3   title         22528 non-null  object        
 4   url           22528 non-null  object        
 5   release_date  22528 non-null  datetime64[ns]
 6   tags          22528 non-null  object        
 7   reviews_url   22528 non-null  object        
 8   specs         22528 non-null  object        
 9   price         22528 non-null  object        
 10  early_access  22528 non-null  int32         
 11  id            22528 non-null  int32         
 12  developer     22528 non-null  object        
dtypes: datetime64[ns](1), int32(2), object(10)
memory usage: 2.2+ MB


In [148]:
df_games_final.to_csv('./DATA/DATA PROCESADA/df_games_final.csv', index=False) # Guardamos el dataframe final en un archivo csv.